# JOINS

- A join brings together two sets of data, the *left* and the *right*.
- A **join expression** determines **whether** rows should join.
- A **join type** determines **what** rows appear in the output.


In [1]:
from pyspark.sql import SparkSession

spark = SparkSession.builder \
    .appName("Joins") \
    .getOrCreate()

spark

Using Spark's default log4j profile: org/apache/spark/log4j2-defaults.properties
25/07/18 20:06:32 WARN Utils: Your hostname, Satkars-MacBook-Pro.local, resolves to a loopback address: 127.0.0.1; using 192.168.0.101 instead (on interface en0)
25/07/18 20:06:32 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Using Spark's default log4j profile: org/apache/spark/log4j2-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/07/18 20:06:32 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
25/07/18 20:06:32 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.


In [3]:
## lets begin by creating some simple datasets to explore the example codes;

person = spark.createDataFrame([
(0, "Bill Chambers", 0, [100]),
(1, "Matei Zaharia", 1, [500, 250, 100]),
(2, "Michael Armbrust", 1, [250, 100])])\
.toDF("id", "name", "graduate_program", "spark_status")
graduateProgram = spark.createDataFrame([
(0, "Masters", "School of Information", "UC Berkeley"),
(2, "Masters", "EECS", "UC Berkeley"),
(1, "Ph.D.", "EECS", "UC Berkeley")])\
.toDF("id", "degree", "department", "school")
sparkStatus = spark.createDataFrame([
(500, "Vice President"),
(250, "PMC Member"),
(100, "Contributor")])\
.toDF("id", "status")

In [5]:
## now lets register these as tables :
person.createOrReplaceTempView("person")
graduateProgram.createOrReplaceTempView("graduateProgram")
sparkStatus.createOrReplaceTempView("sparkStatus")

## Inner Joins

- Keeps **only matching rows** from both sides.


In [7]:
# Define the join condition
# match graduate_program from person with id from graduateProgram
joinExpression = person["graduate_program"] == graduateProgram["id"]

In [9]:
# Perform the join using the default join type (which is 'inner')
person.join(graduateProgram, joinExpression).show()

# Joins person and graduateProgram where the joinExpression is true

#In SQL:
# SELECT * FROM person JOIN graduateProgram
#     ON person.graduate_program = graduateProgram.id

+---+----------------+----------------+---------------+---+-------+--------------------+-----------+
| id|            name|graduate_program|   spark_status| id| degree|          department|     school|
+---+----------------+----------------+---------------+---+-------+--------------------+-----------+
|  0|   Bill Chambers|               0|          [100]|  0|Masters|School of Informa...|UC Berkeley|
|  1|   Matei Zaharia|               1|[500, 250, 100]|  1|  Ph.D.|                EECS|UC Berkeley|
|  2|Michael Armbrust|               1|     [250, 100]|  1|  Ph.D.|                EECS|UC Berkeley|
+---+----------------+----------------+---------------+---+-------+--------------------+-----------+



In [10]:
## we can also do this by passing join type as the 3rd parameter
joinType = "inner"
person.join(graduateProgram, joinExpression, joinType).show()

+---+----------------+----------------+---------------+---+-------+--------------------+-----------+
| id|            name|graduate_program|   spark_status| id| degree|          department|     school|
+---+----------------+----------------+---------------+---+-------+--------------------+-----------+
|  0|   Bill Chambers|               0|          [100]|  0|Masters|School of Informa...|UC Berkeley|
|  1|   Matei Zaharia|               1|[500, 250, 100]|  1|  Ph.D.|                EECS|UC Berkeley|
|  2|Michael Armbrust|               1|     [250, 100]|  1|  Ph.D.|                EECS|UC Berkeley|
+---+----------------+----------------+---------------+---+-------+--------------------+-----------+



## Outer Joins
- Keeps all rows from both sides, fills nulls if no match

In [11]:
joinType = "outer"

person.join(graduateProgram, joinExpression, joinType).show()

+----+----------------+----------------+---------------+---+-------+--------------------+-----------+
|  id|            name|graduate_program|   spark_status| id| degree|          department|     school|
+----+----------------+----------------+---------------+---+-------+--------------------+-----------+
|   0|   Bill Chambers|               0|          [100]|  0|Masters|School of Informa...|UC Berkeley|
|   1|   Matei Zaharia|               1|[500, 250, 100]|  1|  Ph.D.|                EECS|UC Berkeley|
|   2|Michael Armbrust|               1|     [250, 100]|  1|  Ph.D.|                EECS|UC Berkeley|
|NULL|            NULL|            NULL|           NULL|  2|Masters|                EECS|UC Berkeley|
+----+----------------+----------------+---------------+---+-------+--------------------+-----------+



## Left Outer Joins
- Keeps all rows from left, with nulls if no match in right

In [12]:
joinType = "left_outer"

person.join(graduateProgram, joinExpression, joinType).show()

+---+----------------+----------------+---------------+---+-------+--------------------+-----------+
| id|            name|graduate_program|   spark_status| id| degree|          department|     school|
+---+----------------+----------------+---------------+---+-------+--------------------+-----------+
|  0|   Bill Chambers|               0|          [100]|  0|Masters|School of Informa...|UC Berkeley|
|  1|   Matei Zaharia|               1|[500, 250, 100]|  1|  Ph.D.|                EECS|UC Berkeley|
|  2|Michael Armbrust|               1|     [250, 100]|  1|  Ph.D.|                EECS|UC Berkeley|
+---+----------------+----------------+---------------+---+-------+--------------------+-----------+



## Right Outer Joins
- Keeps all rows from right, with nulls if no match in left.

In [13]:
joinType = "right_outer"

person.join(graduateProgram, joinExpression, joinType).show()

+----+----------------+----------------+---------------+---+-------+--------------------+-----------+
|  id|            name|graduate_program|   spark_status| id| degree|          department|     school|
+----+----------------+----------------+---------------+---+-------+--------------------+-----------+
|   0|   Bill Chambers|               0|          [100]|  0|Masters|School of Informa...|UC Berkeley|
|NULL|            NULL|            NULL|           NULL|  2|Masters|                EECS|UC Berkeley|
|   2|Michael Armbrust|               1|     [250, 100]|  1|  Ph.D.|                EECS|UC Berkeley|
|   1|   Matei Zaharia|               1|[500, 250, 100]|  1|  Ph.D.|                EECS|UC Berkeley|
+----+----------------+----------------+---------------+---+-------+--------------------+-----------+



## Left Semi Joins
- Keep rows the left DF that have a match in the right DF.

In [14]:
joinType = "left_semi"

person.join(graduateProgram, joinExpression, joinType).show()

+---+----------------+----------------+---------------+
| id|            name|graduate_program|   spark_status|
+---+----------------+----------------+---------------+
|  0|   Bill Chambers|               0|          [100]|
|  1|   Matei Zaharia|               1|[500, 250, 100]|
|  2|Michael Armbrust|               1|     [250, 100]|
+---+----------------+----------------+---------------+



In [15]:
gradProgram2 = graduateProgram.union(spark.createDataFrame([
    (0, "Masters", "Duplicated Row", "Duplicated School")]))

gradProgram2.createOrReplaceTempView("gradProgram2")

gradProgram2.join(person, joinExpression, joinType).show()

+---+-------+--------------------+-----------------+
| id| degree|          department|           school|
+---+-------+--------------------+-----------------+
|  0|Masters|School of Informa...|      UC Berkeley|
|  1|  Ph.D.|                EECS|      UC Berkeley|
|  0|Masters|      Duplicated Row|Duplicated School|
+---+-------+--------------------+-----------------+



## Left Anti Joins
- The `opposite` of left semi joins where the result retains the rows that have no matches on the RHS.

In [16]:
joinType = "left_anti"
graduateProgram.join(person, joinExpression, joinType).show()

# -- in SQL
# SELECT * FROM graduateProgram LEFT ANTI JOIN person
# ON graduateProgram.id = person.graduate_program

+---+-------+----------+-----------+
| id| degree|department|     school|
+---+-------+----------+-----------+
|  2|Masters|      EECS|UC Berkeley|
+---+-------+----------+-----------+



## Natural Join
- Automatically matches columns with the same names.

## Cross Join
- Cartesian product: every row in left joins every row in right

In [18]:
joinType = "cross"
graduateProgram.join(person, joinExpression, joinType).show()

# -- in SQL
# SELECT * FROM graduateProgram CROSS JOIN person
# ON graduateProgram.id = person.graduate_program

+---+-------+--------------------+-----------+---+----------------+----------------+---------------+
| id| degree|          department|     school| id|            name|graduate_program|   spark_status|
+---+-------+--------------------+-----------+---+----------------+----------------+---------------+
|  0|Masters|School of Informa...|UC Berkeley|  0|   Bill Chambers|               0|          [100]|
|  1|  Ph.D.|                EECS|UC Berkeley|  1|   Matei Zaharia|               1|[500, 250, 100]|
|  1|  Ph.D.|                EECS|UC Berkeley|  2|Michael Armbrust|               1|     [250, 100]|
+---+-------+--------------------+-----------+---+----------------+----------------+---------------+



In [21]:
# Calling the cross-join out explicitly:

person.crossJoin(graduateProgram).show()

+---+----------------+----------------+---------------+---+-------+--------------------+-----------+
| id|            name|graduate_program|   spark_status| id| degree|          department|     school|
+---+----------------+----------------+---------------+---+-------+--------------------+-----------+
|  0|   Bill Chambers|               0|          [100]|  0|Masters|School of Informa...|UC Berkeley|
|  0|   Bill Chambers|               0|          [100]|  2|Masters|                EECS|UC Berkeley|
|  0|   Bill Chambers|               0|          [100]|  1|  Ph.D.|                EECS|UC Berkeley|
|  1|   Matei Zaharia|               1|[500, 250, 100]|  0|Masters|School of Informa...|UC Berkeley|
|  1|   Matei Zaharia|               1|[500, 250, 100]|  2|Masters|                EECS|UC Berkeley|
|  1|   Matei Zaharia|               1|[500, 250, 100]|  1|  Ph.D.|                EECS|UC Berkeley|
|  2|Michael Armbrust|               1|     [250, 100]|  0|Masters|School of Informa...|UC 

## Joins on Complex Type

- Wehn one column is an array and the other is a scalar, we can join them by using `array_contains`

In [23]:
from pyspark.sql.functions import expr

person.withColumnRenamed("id", "personId") \
    .join(sparkStatus, expr("array_contains(spark_status, id)")).show()

# -- in SQL
# SELECT * FROM
# (select id as personId, name, graduate_program, spark_status FROM person)
# INNER JOIN sparkStatus ON array_contains(spark_status, id)

+--------+----------------+----------------+---------------+---+--------------+
|personId|            name|graduate_program|   spark_status| id|        status|
+--------+----------------+----------------+---------------+---+--------------+
|       0|   Bill Chambers|               0|          [100]|100|   Contributor|
|       1|   Matei Zaharia|               1|[500, 250, 100]|500|Vice President|
|       1|   Matei Zaharia|               1|[500, 250, 100]|250|    PMC Member|
|       1|   Matei Zaharia|               1|[500, 250, 100]|100|   Contributor|
|       2|Michael Armbrust|               1|     [250, 100]|250|    PMC Member|
|       2|Michael Armbrust|               1|     [250, 100]|100|   Contributor|
+--------+----------------+----------------+---------------+---+--------------+



## How Spark Performs Joins

To core components `technical` are the **node-to-node communication straegy** and **per node computation strategy**

Further, Spark has two main **communication strategies** for joins:

**1. Shuffle Join:** 

- Each node sends data to **all other nodes**.
- This is **expensive** because it causes **network-wide communication.**
- Happens when **both tables are large**, and no optimization (like paritioning or broadcasting) is possible.

**2. Broadcast Join:** 

- Spark **sends the smaller table to every node** in the cluster.
- Much **faster and cheaper** than shuffle joins.
- Only works when one of the tables is **small enough to fit in memory.**

In [24]:
person.join(graduateProgram, joinExpression).explain()

== Physical Plan ==
AdaptiveSparkPlan isFinalPlan=false
+- SortMergeJoin [graduate_program#6L], [id#12L], Inner
   :- Sort [graduate_program#6L ASC NULLS FIRST], false, 0
   :  +- Exchange hashpartitioning(graduate_program#6L, 200), ENSURE_REQUIREMENTS, [plan_id=1696]
   :     +- Project [_1#0L AS id#4L, _2#1 AS name#5, _3#2L AS graduate_program#6L, _4#3 AS spark_status#7]
   :        +- Filter isnotnull(_3#2L)
   :           +- Scan ExistingRDD[_1#0L,_2#1,_3#2L,_4#3]
   +- Sort [id#12L ASC NULLS FIRST], false, 0
      +- Exchange hashpartitioning(id#12L, 200), ENSURE_REQUIREMENTS, [plan_id=1697]
         +- Project [_1#8L AS id#12L, _2#9 AS degree#13, _3#10 AS department#14, _4#11 AS school#15]
            +- Filter isnotnull(_1#8L)
               +- Scan ExistingRDD[_1#8L,_2#9,_3#10,_4#11]


